In [7]:
%pip install langchain langchain_openai langchain-community --upgrade

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)

In [9]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

print(runnable.invoke(1)) # {'origin': 1, 'modified': 2}


def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return prompt + " world"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] # Parsing logic
}

chain.invoke('hello') 

{'origin': 1, 'modified': 2}


{'original': 'hello world', 'parsed': 'dlrow olleh'}

PROMPT + MODEL

In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

chat = ChatOpenAI(model="gpt-4o-mini")
prompt = ChatPromptTemplate.from_template('Tell me a joke about {topic}')

chain = prompt | chat
print(chain)

first=ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})]) middle=[] last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000163639EAD50>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000163639EE5D0>, root_client=<openai.OpenAI object at 0x000001637EA16490>, root_async_client=<openai.AsyncOpenAI object at 0x00000163639EAE90>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))


In [11]:
print("first", chain.first)
print("last", chain.last)

first input_variables=['topic'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})]
last client=<openai.resources.chat.completions.completions.Completions object at 0x00000163639EAD50> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000163639EE5D0> root_client=<openai.OpenAI object at 0x000001637EA16490> root_async_client=<openai.AsyncOpenAI object at 0x00000163639EAE90> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')


In [ ]:
# Stream:
print('\n\nStream:\n')
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

# Invoke:
print('\n\nInvoke:\n')
print(chain.invoke({"topic": "bears"}).content)

# Batch: 
print('\n\nBatch:\n')
print(chain.batch([{"topic": "bears"}, {"topic": "cats"}, {"topic": "dogs"}]))
#this is the most const effective way to use the API. Multiple prompts are sent in a single request and multiple responses are returned in a single response.



Stream:

Why do bears never use cell phones?

Because they always get stuck in "bear" reception!

Invoke:

Why do bears have hairy coats?

Because they look silly in sweaters!


Batch:

[AIMessage(content='Why did the bear sit on a log?\n\nBecause it wanted to log its hours!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 13, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3267753c5d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd458130-892e-483b-ae50-7fb1789f00e3-0', usage_metadata={'input_tokens': 13, 'output_tokens': 18, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content

Retrieval Augmented Generation (RAG) in LCEL

In [ ]:
%pip install langchain openai faiss-cpu tiktoken --upgrade --quiet

In [15]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores.faiss import FAISS

In [24]:
#Create a Facebook AI Similarity Search (FAISS) vector store
vectorstore = FAISS.from_texts(
    ["Marcell Krausz is learning data engineering and LLM at home.", "Marcell has an age of 31 years old."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
#minimize the hallucination by using the context
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model 
    | StrOutputParser() #parse the output
)

# It's the same as this, but the tuple allows for line breaks:
# {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

In [26]:
chain.invoke("What does Marcell learn at home?")

'Marcell is learning data engineering and LLM at home.'

In [23]:
chain.invoke("What is Marcell's age?")

"Marcell's age is 31 years old."

In [28]:
# How itemgetter works
test = {
    "data": ['This is a test', 'Another entry...']
}

print(itemgetter(test))
print(itemgetter('data')(test))

operator.itemgetter({'data': ['This is a test', 'Another entry...']})
['This is a test', 'Another entry...']


In [30]:
prompt = ChatPromptTemplate.from_template('''What is the profession of Marcell Krausz? His profession is {profession}.''')

first_chain = RunnableParallel(
    name=lambda x: "Marcell Krausz",
    age=lambda x: 31
)

second_chain = {
    # itemgetter is used to get the value from the dictionary from the previous step: (note this is only the previous step, not the whole chain)
    'name': itemgetter('name'),
    'age': itemgetter('age'),
    # You can not use string values, either use itemgetter or a lambda, or RunnablePassthrough
    'profession': lambda x: "Java Engineer"
}

chain = first_chain | second_chain |  prompt |  ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
chain.invoke({})

'Marcell Krausz is a Java Engineer.'